In [1]:
# 1. Prepare dependencies
import mnist

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn


In [2]:
# 2. prepare training and testing dataloader

mnist.datasets_url="http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()


train_images_tensor = torch.Tensor(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.int64)
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.int64)

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [4]:
# Define hyperparameters

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# Define the train_loop function

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        
        
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')








In [6]:
loss_fn = nn.CrossEntropyLoss()

optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)


epochs = 10

for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)


Epoch 1
--------------------
loss: 5.904955 [   64/60000]
loss: 2.028247 [ 6464/60000]
loss: 1.575328 [12864/60000]
loss: 1.505259 [19264/60000]
loss: 1.558687 [25664/60000]
loss: 1.485648 [32064/60000]
loss: 1.493321 [38464/60000]
loss: 1.616504 [44864/60000]
loss: 0.983384 [51264/60000]
loss: 0.856178 [57664/60000]
Epoch 2
--------------------
loss: 0.994400 [   64/60000]
loss: 0.897829 [ 6464/60000]
loss: 0.888292 [12864/60000]
loss: 0.689002 [19264/60000]
loss: 0.740498 [25664/60000]
loss: 0.719097 [32064/60000]
loss: 0.436040 [38464/60000]
loss: 0.708937 [44864/60000]
loss: 0.637546 [51264/60000]
loss: 0.528562 [57664/60000]
Epoch 3
--------------------
loss: 0.401631 [   64/60000]
loss: 0.211469 [ 6464/60000]
loss: 0.358625 [12864/60000]
loss: 0.275176 [19264/60000]
loss: 0.219800 [25664/60000]
loss: 0.464298 [32064/60000]
loss: 0.092957 [38464/60000]
loss: 0.250099 [44864/60000]
loss: 0.476184 [51264/60000]
loss: 0.361490 [57664/60000]
Epoch 4
--------------------
loss: 0.190738